In [1]:
import glob
import os
import tensorflow as tf

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # Dùng để chia train/validation
from sklearn.utils import shuffle

In [2]:
path_csv = 'D:/AI/Chess/ChessApp/Chess/csv_data/morphy'
# Ví dụ nếu bạn tải lên Colab:
# path_csv = '/content/csv_data/morphy'

if not os.path.isdir(path_csv):
    print(f"Lỗi: Thư mục không tồn tại: {path_csv}")
    print("Vui lòng kiểm tra lại đường dẫn `path_csv`.")
    # Dừng thực thi nếu thư mục không đúng (trong môi trường notebook có thể không dừng hẳn)
    raise FileNotFoundError(f"Thư mục không tồn tại: {path_csv}")
else:
    files_csv = glob.glob(os.path.join(path_csv, "*.csv")) # Dùng os.path.join cho an toàn

    if not files_csv:
        print(f"Lỗi: Không tìm thấy file CSV nào trong: {path_csv}")
        raise FileNotFoundError(f"Không tìm thấy file CSV nào trong: {path_csv}")
    else:
        li = []
        print(f"Đang đọc {len(files_csv)} file CSV từ {path_csv}...")
        for filename in files_csv:
            try:
                df = pd.read_csv(filename, index_col=None, header=0)
                li.append(df)
            except Exception as e:
                print(f"Lỗi khi đọc file {filename}: {e}")

        if not li:
             print(f"Lỗi: Không đọc được dữ liệu từ bất kỳ file CSV nào.")
             raise ValueError("Không có dữ liệu để xử lý.")
        else:
            train_df = pd.concat(li, axis=0, ignore_index=True)
            print("Đọc dữ liệu hoàn tất.")

Đang đọc 193 file CSV từ D:/AI/Chess/ChessApp/Chess/csv_data/morphy...
Đọc dữ liệu hoàn tất.


In [3]:
train_df = shuffle(train_df, random_state=42)

In [4]:
train_df.shape

(213027, 193)

In [5]:
# Xác định cột trạng thái bàn cờ (giả định 64 cột đầu) và cột nước đi
board_columns = list(train_df.iloc[:, 0:64].columns)
move_columns = list(train_df.iloc[:, 64:192].columns) # Kiểm tra lại index nếu cần
label_column = 'good_move'

# Xử lý NaN cho cột trạng thái bàn cờ (categorical) -> thay bằng 'EMPTY'
train_df[board_columns] = train_df[board_columns].fillna('EMPTY')
print(f"Đã xử lý NaN cho {len(board_columns)} cột trạng thái bàn cờ.")

# Xử lý NaN cho cột nước đi (numerical) -> thay bằng 0.0
# Kiểm tra kiểu dữ liệu trước khi fillna
numeric_move_cols = train_df[move_columns].select_dtypes(include=np.number).columns
non_numeric_move_cols = train_df[move_columns].select_dtypes(exclude=np.number).columns

if len(non_numeric_move_cols) > 0:
    print(f"Cảnh báo: Các cột nước đi sau không phải kiểu số: {list(non_numeric_move_cols)}")
    # Quyết định cách xử lý (ví dụ: cố gắng chuyển đổi hoặc bỏ qua)
    # Ở đây ta thử chuyển đổi, nếu lỗi thì fill bằng 0
    for col in non_numeric_move_cols:
        try:
            train_df[col] = pd.to_numeric(train_df[col], errors='coerce') # Chuyển thành NaN nếu không phải số
        except Exception as e:
             print(f"Không thể chuyển cột {col} thành số: {e}")
    # Sau khi coerce, tất cả cột số hoặc NaN sẽ được fill bằng 0.0
    train_df[move_columns] = train_df[move_columns].fillna(0.0)
else:
    train_df[move_columns] = train_df[move_columns].fillna(0.0)

print(f"Đã xử lý NaN cho {len(move_columns)} cột nước đi (thay bằng 0.0).")

# Kiểm tra lại xem còn NaN không
print("Số lượng NaN còn lại:", train_df.isnull().sum().sum())


Đã xử lý NaN cho 64 cột trạng thái bàn cờ.
Đã xử lý NaN cho 128 cột nước đi (thay bằng 0.0).
Số lượng NaN còn lại: 0


In [6]:
train_df.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
129786,bR,bN,bB,bQ,bK,bB,bN,bR,bp,bp,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
190683,bR,--,--,bQ,bK,bB,bN,bR,bp,bB,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
89746,bR,--,bB,--,bR,--,bK,--,bp,bp,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
122470,bR,--,bB,bQ,--,bR,bK,--,bp,bp,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
17953,bR,--,bB,--,bK,--,bN,bR,bp,bp,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [7]:
features = list(train_df.iloc[:, 0:192].columns)
categorical_features = board_columns # Đã xác định ở trên
numerical_features = move_columns   # Đã xác định ở trên

In [8]:
X = train_df[features]
y = train_df[label_column]

In [9]:
y = y.astype(int)

print("\nKiểu dữ liệu của X:", type(X))
print("Kích thước của X:", X.shape)
print("Kiểu dữ liệu của y:", type(y))
print("Kích thước của y:", y.shape)
print("Các giá trị duy nhất trong y:", y.unique())

# Chia dữ liệu thành tập huấn luyện và tập validation (ví dụ: 80% train, 20% val)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nKích thước tập huấn luyện (X): {X_train.shape}")
print(f"Kích thước tập huấn luyện (y): {y_train.shape}")
print(f"Kích thước tập validation (X): {X_val.shape}")
print(f"Kích thước tập validation (y): {y_val.shape}")


Kiểu dữ liệu của X: <class 'pandas.core.frame.DataFrame'>
Kích thước của X: (213027, 192)
Kiểu dữ liệu của y: <class 'pandas.core.series.Series'>
Kích thước của y: (213027,)
Các giá trị duy nhất trong y: [0 1]

Kích thước tập huấn luyện (X): (170421, 192)
Kích thước tập huấn luyện (y): (170421,)
Kích thước tập validation (X): (42606, 192)
Kích thước tập validation (y): (42606,)


In [10]:

BATCH_SIZE = 256 # Tăng batch size có thể tăng tốc độ huấn luyện
BUFFER_SIZE = 10000 # Kích thước buffer để shuffle

# Hàm tiện ích để tạo Dataset từ Pandas DataFrame
def df_to_dataset(dataframe_X, dataframe_y, shuffle=True, batch_size=BATCH_SIZE):
  # Chuyển đổi thành từ điển các tensors
  dict_slices = dict(dataframe_X)
  # Tạo dataset từ các slices tensor
  ds = tf.data.Dataset.from_tensor_slices((dict_slices, dataframe_y))
  if shuffle:
    ds = ds.shuffle(buffer_size=BUFFER_SIZE)
  ds = ds.batch(batch_size)
  # Prefetch để tối ưu hiệu năng I/O
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds
     

In [11]:
train_ds = df_to_dataset(X_train, y_train)
val_ds = df_to_dataset(X_val, y_val, shuffle=False)

In [ ]:
# batches_X, batches_y = split_into_batches(train)

In [13]:
feature_inputs = []
encoded_features = []

# 1. Xử lý Đặc trưng hạng mục (Categorical - Board State)
print("\nĐịnh nghĩa lớp tiền xử lý cho đặc trưng hạng mục:")
for feature_name in categorical_features:
  # Tạo Input layer cho đặc trưng này (kiểu string)
  cat_input = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.string)
  feature_inputs.append(cat_input)

  # Tìm bộ từ vựng (vocabulary) duy nhất từ dữ liệu huấn luyện
  vocabulary = X_train[feature_name].unique()
  print(f"  - {feature_name}: Vocabulary size = {len(vocabulary)}")

  # StringLookup: Chuyển đổi string thành index số nguyên
  string_lookup = tf.keras.layers.StringLookup(
      vocabulary=vocabulary, output_mode='int', name=f'{feature_name}_lookup'
  )

  # CategoryEncoding (OneHot): Chuyển index thành one-hot vector
  # Hoặc dùng Embedding: layers.Embedding(input_dim=len(vocabulary) + 1, output_dim=...)
  one_hot_encoder = tf.keras.layers.CategoryEncoding(
      num_tokens=len(vocabulary), output_mode='one_hot', name=f'{feature_name}_onehot'
  )

  # Kết nối các lớp xử lý
  encoded = string_lookup(cat_input)
  encoded = one_hot_encoder(encoded)
  encoded_features.append(encoded)

# 2. Xử lý Đặc trưng số (Numerical - Move Info)
print("\nĐịnh nghĩa lớp tiền xử lý cho đặc trưng số:")
for feature_name in numerical_features:
  # Tạo Input layer cho đặc trưng này (kiểu float32)
  num_input = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.float32)
  feature_inputs.append(num_input)

  # Normalization layer (tùy chọn nhưng thường hữu ích)
  # Cần 'adapt' layer này trên dữ liệu huấn luyện
  normalizer = tf.keras.layers.Normalization(name=f'{feature_name}_norm')
  # Lấy dữ liệu của cột này từ X_train để adapt
  feature_data = X_train[feature_name].values.reshape(-1, 1)
  normalizer.adapt(feature_data)
  print(f"  - {feature_name}: Normalized.")

  # Kết nối lớp xử lý
  encoded = normalizer(num_input)
  encoded_features.append(encoded)


Định nghĩa lớp tiền xử lý cho đặc trưng hạng mục:
  - a1: Vocabulary size = 7
  - b1: Vocabulary size = 8
  - c1: Vocabulary size = 11
  - d1: Vocabulary size = 9
  - e1: Vocabulary size = 9
  - f1: Vocabulary size = 10
  - g1: Vocabulary size = 10
  - h1: Vocabulary size = 7
  - a2: Vocabulary size = 10
  - b2: Vocabulary size = 13
  - c2: Vocabulary size = 13
  - d2: Vocabulary size = 13
  - e2: Vocabulary size = 13
  - f2: Vocabulary size = 12
  - g2: Vocabulary size = 12
  - h2: Vocabulary size = 12
  - a3: Vocabulary size = 12
  - b3: Vocabulary size = 13
  - c3: Vocabulary size = 13
  - d3: Vocabulary size = 13
  - e3: Vocabulary size = 13
  - f3: Vocabulary size = 13
  - g3: Vocabulary size = 13
  - h3: Vocabulary size = 12
  - a4: Vocabulary size = 12
  - b4: Vocabulary size = 13
  - c4: Vocabulary size = 12
  - d4: Vocabulary size = 13
  - e4: Vocabulary size = 13
  - f4: Vocabulary size = 13
  - g4: Vocabulary size = 13
  - h4: Vocabulary size = 13
  - a5: Vocabulary size = 

In [14]:
# Ghép nối tất cả các đặc trưng đã được xử lý
all_features_concatenated = tf.keras.layers.Concatenate(name='feature_concat')(encoded_features)

# Thêm các lớp Dense (Fully Connected)
# Lớp ẩn đầu tiên
x = tf.keras.layers.Dense(128, activation='relu', name='hidden_layer_1')(all_features_concatenated)
x = tf.keras.layers.Dropout(0.3, name='dropout_1')(x) # Thêm Dropout để chống overfitting
# Lớp ẩn thứ hai (tùy chọn)
x = tf.keras.layers.Dense(64, activation='relu', name='hidden_layer_2')(x)
x = tf.keras.layers.Dropout(0.3, name='dropout_2')(x)

# Lớp Output cho bài toán phân loại nhị phân
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(x)

# Tạo mô hình Keras
model = tf.keras.Model(inputs=feature_inputs, outputs=output, name='chess_move_predictor')

# In cấu trúc mô hình
model.summary()

Model: "chess_move_predictor"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ a1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ c1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ d1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ e1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ f1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ g1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ h1 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ a2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ c2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ d2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ e2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ f2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ g2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ h2 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ a3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ c3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ d3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ e3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ f3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ g3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ h3 (InputLayer)     │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ a4 (InputLayer)     │ (None, 1)         │          0 │ -               

 Total params: 120,321 (470.50 KB)

 Trainable params: 119,937 (468.50 KB)

 Non-trainable params: 384 (2.00 KB)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # Adam là lựa chọn phổ biến
    loss='binary_crossentropy',      # Loss function cho phân loại nhị phân
    metrics=['accuracy']             # Theo dõi độ chính xác
)

print("\nĐã biên dịch mô hình Keras.")

# @title Cell 13: Train Model (Thay thế vòng lặp estimator.train)

EPOCHS = 10 # Số lần duyệt qua toàn bộ tập huấn luyện

print(f"\nBắt đầu huấn luyện mô hình trong {EPOCHS} epochs...")

# Thêm EarlyStopping để dừng huấn luyện sớm nếu không cải thiện
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # Theo dõi loss trên tập validation
    patience=3,         # Dừng sau 3 epochs không cải thiện
    restore_best_weights=True # Khôi phục trọng số tốt nhất
)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping] # Thêm callback vào huấn luyện
)

print("\nHoàn tất huấn luyện.")

# Đánh giá mô hình trên tập validation
loss, accuracy = model.evaluate(val_ds)
print(f"\nValidation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Đã biên dịch mô hình Keras.

Bắt đầu huấn luyện mô hình trong 10 epochs...
Epoch 1/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 25s 31ms/step - accuracy: 0.9584 - loss: 0.1599 - val_accuracy: 0.9707 - val_loss: 0.1222
Epoch 2/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.9716 - loss: 0.1243 - val_accuracy: 0.9715 - val_loss: 0.1201
Epoch 3/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.9718 - loss: 0.1206 - val_accuracy: 0.9720 - val_loss: 0.1169
Epoch 4/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.9721 - loss: 0.1163 - val_accuracy: 0.9722 - val_loss: 0.1167
Epoch 5/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.9720 - loss: 0.1126 - val_accuracy: 0.9727 - val_loss: 0.1170
Epoch 6/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.9720 - loss: 0.1084 - val_accuracy: 0.9726 - val_loss: 0.1173
Epoch 7/10
666/666 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.9724 - loss: 0.1044 - val_accuracy: 0.9728 - val_loss: 0.1179

Hoàn tất huấn

In [19]:
# train the model on all the input functions

# !!! THAY ĐỔI ĐƯỜNG DẪN LƯU CHO PHÙ HỢP !!!
model_save_path = 'D:/AI/Chess/ChessApp/Chess/estimator/morphy'
# Ví dụ nếu bạn dùng Colab:
# model_save_path = '/content/morphy_keras_model'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Lưu mô hình Keras dưới định dạng SavedModel (khuyến nghị)
model.export(model_save_path)

print(f"Đã lưu mô hình Keras thành công vào thư mục: {model_save_path}")
print("Thư mục này bây giờ sẽ chứa 'saved_model.pb' và thư mục con 'variables'.")

# (Tùy chọn) Kiểm tra nội dung thư mục đã tạo
print("\nNội dung thư mục đã lưu:")
try:
  for item in os.listdir(model_save_path):
    item_path = os.path.join(model_save_path, item)
    if os.path.isdir(item_path):
        print(f"- {item}/")
        # In nội dung thư mục con 'variables'
        if item == 'variables':
            for sub_item in os.listdir(item_path):
                 print(f"  - {sub_item}")
    else:
        print(f"- {item}")
except FileNotFoundError:
  print(f"Lỗi: Không tìm thấy thư mục {model_save_path}")
except Exception as e:
    print(f"Lỗi khi liệt kê thư mục: {e}")

INFO:tensorflow:Assets written to: D:/AI/Chess/ChessApp/Chess/estimator/morphy\assets


INFO:tensorflow:Assets written to: D:/AI/Chess/ChessApp/Chess/estimator/morphy\assets


Saved artifact at 'D:/AI/Chess/ChessApp/Chess/estimator/morphy'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 1), dtype=tf.string, name='a1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='b1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='c1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='d1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='e1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='f1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='g1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='h1'), TensorSpec(shape=(None, 1), dtype=tf.string, name='a2'), TensorSpec(shape=(None, 1), dtype=tf.string, name='b2'), TensorSpec(shape=(None, 1), dtype=tf.string, name='c2'), TensorSpec(shape=(None, 1), dtype=tf.string, name='d2'), TensorSpec(shape=(None, 1), dtype=tf.string, name='e2'), TensorSpec(shape=(None, 1), dtype=tf.string, name='f2'), TensorSpec(shape=(None, 1), dtype=tf.string, 